# Introduction
### According to new data from the restaurant reviewing website Yelp, the share of Chinese restaurants in the top 20 metropolitan areas has been consistently falling. In 2015, an average of 7.3 percent of all restaurants in these areas were Chinese, compared with 6.5 percent in 2019. The share of Chinese restaurants in NYC has been declined just a little bit. In 2015, the figure was about 9 percent and in 2019, the figure became 8.6 percent. The decline in NYC is not obvious compared to the average decline in the top 20 metropolitan areas. Therefore, we can assume the demand of Chinese cuisine is still strong in NYC.

### The purpose of this project is to explore which neighborhood in New York City has less density of Chinese restaurant.

# Data sources
### We use the data from Foursquare.com to explore the density of Chinese restaurant in each neighborhood. In principle, the lower the density is, the lower the competition is. However, the result is for reference only because those who want to open a restaurant should also consider other factors, like the neighborhood's demographic, the average household income, and the security level, etc.

In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json # library to handle JSON files
!conda install -c conda-forge geopy --yes # install the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
!conda install -c conda-forge folium=0.5.0 --yes # install the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: - 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - conda-forge/linux-64::pytorch==1.8.0=cpu_py37hafa7651_0
  - defaults/noarch::ibm-wsrt-py37main-keep==0.0.0=1937
  - defaults/noarch::ibm-wsrt-py37main-main==custom=1937
done

# All requested packages already installed.

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: \ 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - conda-forge/linux-64::pytorch==1.8.0=cpu_py37hafa7651_0
  - defaults/noarch::ibm-wsrt-py37main-keep==0.0.0=1937
  - defaults/noarch::ibm-wsrt-py37main-main==custom=1937
done

## Package Plan ##

  environment location: /opt/conda/envs/Python-3.7-main

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |         

In [2]:
# download and load the dataset of NYC neighborhood
!wget -q -O 'newyork_data.json' https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs/newyork_data.json

with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

## Tidy Data

In [3]:
# define a new variable that includes feature key
neighborhoods_data = newyork_data['features']

# Tranform the data into a pandas dataframe
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

#loop through the data and fill the dataframe one row at a time
for data in neighborhoods_data:
    #borough = neighborhood_name = data['properties']['borough'] 
    borough = data['properties']['borough'] # data is defined in the beginning of the loop
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [4]:
print(neighborhoods.shape)
neighborhoods.head()

(306, 4)


,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [5]:
# make sure that the dataset has all 5 boroughs and 306 neighborhoods.
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


## Acquire NYC data from Foursquare.com

In [6]:
CLIENT_ID = '0WZQC5ZYCJ3K2HJE1XMCSPPYY24TZKU5LA05I0L0A0UHVHIY' # your Foursquare ID
CLIENT_SECRET = 'MAJCBSZR4SAHGL2HNFERENSNIAWRIYL3RX5STMLEM2BQXA0F' # your Foursquare Secret
ACCESS_TOKEN = 'QOEG0C2TDJXPKLRGZSYH5S2RYC2NGEHGUAH3KFUGWHPDD2DN' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 0WZQC5ZYCJ3K2HJE1XMCSPPYY24TZKU5LA05I0L0A0UHVHIY
CLIENT_SECRET:MAJCBSZR4SAHGL2HNFERENSNIAWRIYL3RX5STMLEM2BQXA0F


## Get the latitude and longitude of NYC

In [7]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


## Create a map of New York with neighborhoods superimposed on top

In [8]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [ ]:
neighborhoods

## Create a function to collect venues data to all the neighborhoods in NYC

In [9]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [10]:
# Now write the code to run the above function on each neighborhood and create a new dataframe called NYC_venues.
NYC_venues = getNearbyVenues(names = neighborhoods['Neighborhood'],
                                 latitudes = neighborhoods['Latitude'],
                                 longitudes = neighborhoods['Longitude'])

Wakefield
Co-op City
Eastchester
Fieldston
Riverdale
Kingsbridge
Marble Hill
Woodlawn
Norwood
Williamsbridge
Baychester
Pelham Parkway
City Island
Bedford Park
University Heights
Morris Heights
Fordham
East Tremont
West Farms
High  Bridge
Melrose
Mott Haven
Port Morris
Longwood
Hunts Point
Morrisania
Soundview
Clason Point
Throgs Neck
Country Club
Parkchester
Westchester Square
Van Nest
Morris Park
Belmont
Spuyten Duyvil
North Riverdale
Pelham Bay
Schuylerville
Edgewater Park
Castle Hill
Olinville
Pelham Gardens
Concourse
Unionport
Edenwald
Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker

In [11]:
print(NYC_venues.shape)
#NYC_venues.shape
NYC_venues.head()

(6123, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop
1,Wakefield,40.894705,-73.847201,Rite Aid,40.896649,-73.844846,Pharmacy
2,Wakefield,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop
3,Wakefield,40.894705,-73.847201,Walgreens,40.896528,-73.844700,Pharmacy
4,Wakefield,40.894705,-73.847201,Subway,40.890468,-73.849152,Sandwich Place


## Analyze Each Neighborhood

In [12]:
NYC_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Allerton,30,30,30,30,30,30
Annadale,8,8,8,8,8,8
Arden Heights,5,5,5,5,5,5
Arlington,4,4,4,4,4,4
Arrochar,21,21,21,21,21,21
Arverne,18,18,18,18,18,18
Astoria,30,30,30,30,30,30
Astoria Heights,12,12,12,12,12,12
Auburndale,19,19,19,19,19,19


In [13]:
# one hot encoding
nyc_onehot = pd.get_dummies(NYC_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
nyc_onehot['Neighborhood'] = NYC_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [nyc_onehot.columns[-1]] + list(nyc_onehot.columns[:-1])
nyc_onehot = nyc_onehot[fixed_columns]

In [75]:
nyc_onehot.shape
#nyc_onehot.head()

(6123, 383)

In [18]:
# Next, let's group rows by neighborhood and by taking the sum of the frequency of occurrence of each category
nyc_grouped = nyc_onehot.groupby('Neighborhood').sum().reset_index()
nyc_grouped.tail()

,Neighborhood,Yoga Studio,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Terminal,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beach Bar,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Check Cashing Service,Cheese Shop,Child Care Service,Chinese Restaurant,Christmas Market,Church,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Basketball Court,College Cafeteria,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cooking School,Cosmetics Shop,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dosa Place,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Entertainment Service,Escape Room,Ethiopian Restaurant,Event Service,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Hill,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Library,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Medical Supply Store,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Nightlife Spot,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoor Sculpture,Outdoors & Recreation,Outlet Mall,Outlet Store,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Pie Shop,Pier,Pilates Studio,Pizza Place,Platform,P

In [19]:
# Keep only Chinese Restaurant
nyc_grouped_1 = nyc_grouped.filter(['Neighborhood','Chinese Restaurant'], axis = 1)

In [76]:
nyc_grouped_1.shape

(301, 3)

## Cluster Neighborhoods

In [103]:
# set number of clusters
kclusters = 5
nyc_grouped_clustering = nyc_grouped_1.drop(columns = 'Neighborhood')

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(nyc_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 1, 1, 1, 1, 1, 1, 0, 1, 3], dtype=int32)

In [104]:
#kmeans.labels_.astype(int)
kmeans.labels_.shape

(301,)

In [109]:
# add clustering labels
nyc_grouped_1.insert(0, 'Cluster Labels', kmeans.labels_)

nyc_merged = neighborhoods

# merge nyc_grouped with neighborhoods to add latitude/longitude for each neighborhood
nyc_merged = nyc_merged.join(nyc_grouped_1.set_index('Neighborhood'), on='Neighborhood')

nyc_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,Chinese Restaurant
0,Bronx,Wakefield,40.894705,-73.847201,1.0,0.0
1,Bronx,Co-op City,40.874294,-73.829939,1.0,0.0
2,Bronx,Eastchester,40.887556,-73.827806,0.0,1.0
3,Bronx,Fieldston,40.895437,-73.905643,1.0,0.0
4,Bronx,Riverdale,40.890834,-73.912585,1.0,0.0


In [108]:
#### TEMPORARY USE ####
#nyc_merged.drop(columns = 'Cluster Labels', inplace = True)
#nyc_grouped_1.drop(columns = 'Cluster Labels', inplace = True)
#nyc_merged.insert(0, 'Cluster Labels', kmeans.labels_)
#nyc_merged.head()

In [111]:
nyc_merged['Cluster Labels'].fillna(value = 0, inplace = True)
nyc_merged['Cluster Labels'] = nyc_merged['Cluster Labels'].astype(int)

In [112]:
nyc_merged.groupby('Cluster Labels').mean()

,Latitude,Longitude,Chinese Restaurant
Cluster Labels,,,
0,40.715100,-73.928818,1.0
1,40.690964,-73.958574,0.0
2,40.735289,-73.871284,3.0
3,40.738341,-73.927994,2.0
4,40.678314,-73.875065,4.5


In [114]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(nyc_merged['Latitude'], nyc_merged['Longitude'], nyc_merged['Neighborhood'], nyc_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' cluster' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=1,
        popup=label,
        color=rainbow[cluster - 1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## There is no Chinese restaurant in cluster labels #0

In [115]:
nyc_merged.groupby(by=['Borough']).mean().sort_values(by = 'Chinese Restaurant')

,Latitude,Longitude,Cluster Labels,Chinese Restaurant
Borough,,,,
Staten Island,40.588851,-74.137414,0.841270,0.274194
Manhattan,40.762438,-73.975329,0.925000,0.375000
Brooklyn,40.651089,-73.952999,1.000000,0.457143
Queens,40.706424,-73.824131,0.938272,0.641975
Bronx,40.850482,-73.873442,1.019231,0.673077


In [91]:
nyc_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,Chinese Restaurant
0,Bronx,Wakefield,40.894705,-73.847201,0,0.0
1,Bronx,Co-op City,40.874294,-73.829939,0,0.0
2,Bronx,Eastchester,40.887556,-73.827806,7,1.0
3,Bronx,Fieldston,40.895437,-73.905643,0,0.0
4,Bronx,Riverdale,40.890834,-73.912585,0,0.0


## Which neighboorhoods do not have Chinese restaurant? 

In [102]:
nyc_merged.loc[nyc_merged['Cluster Labels'] == 0, nyc_merged.columns[[1] + list(range(5, nyc_merged.shape[1]))]]

,Neighborhood,Chinese Restaurant
0,Wakefield,0.0
1,Co-op City,0.0
3,Fieldston,0.0
4,Riverdale,0.0
5,Kingsbridge,0.0
6,Marble Hill,0.0
7,Woodlawn,0.0
9,Williamsbridge,0.0
10,Baychester,0.0
12,City Island,0.0


## Which neighborhoods have the lowest density of Chinese restaurant?

In [93]:
nyc_merged.loc[nyc_merged['Cluster Labels'] == 7, nyc_merged.columns[[1] + list(range(5, nyc_merged.shape[1]))]]

,Neighborhood,Chinese Restaurant
13,Bedford Park,3.0
26,Soundview,3.0
50,Gravesend,3.0
100,Chinatown,3.0
153,Little Neck,3.0
170,Far Rockaway,3.0
184,Queensboro Hill,3.0
186,Ravenswood,3.0


In [94]:
nyc_bar_Borough = nyc_merged.filter( ['Borough', 'Chinese Restaurant'], axis = 1)
nyc_bar_Cluster = nyc_merged.filter( ['Cluster Labels', 'Chinese Restaurant'], axis = 1)
nyc_bar_Borough

,Borough,Chinese Restaurant
0,Bronx,0.0
1,Bronx,0.0
2,Bronx,1.0
3,Bronx,0.0
4,Bronx,0.0
5,Bronx,0.0
6,Manhattan,0.0
7,Bronx,0.0
8,Bronx,1.0
9,Bronx,0.0


In [95]:
nyc_bar_Cluster_ = nyc_bar_Cluster.groupby(by=['Cluster Labels']).sum()
nyc_bar_Cluster_
#nyc_merged.groupby(by=['Borough']).mean().sort_values(by = 'Chinese Restaurant')

,Chinese Restaurant
Cluster Labels,
0,0.0
1,24.0
4,5.0
5,4.0
6,44.0
7,74.0


In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt

nyc_bar_Cluster_.plot(kind = 'bar')

In [ ]:
nyc_bar_Borough_ = nyc_bar_Borough.groupby(by=['Borough']).sum()
nyc_bar_Borough_

In [ ]:
nyc_bar_Borough_.plot(kind = 'bar')

In [ ]:
nyc_Manhattan = nyc_merged[nyc_merged['Borough'] =='Manhattan']
nyc_Manhattan.head()

In [ ]:
nyc_Manhattan_1 = nyc_Manhattan.filter( ['Borough', 'Neighborhood','Chinese Restaurant'], axis = 1)
nyc_Manhattan_2 = nyc_Manhattan_1.groupby(by=['Neighborhood']).sum()
nyc_Manhattan_2.head()

In [ ]:
nyc_Manhattan_2.plot(kind = 'bar')

In [ ]:
#from sklearn.preprocessing import MinMaxScaler

In [ ]:
features = nyc_merged[['Latitude', 'Longitude', "Chinese Restaurant"]]
x = features.values
#min_max_scaler = MinMaxScaler()
#feature_mtx = min_max_scaler.fit_transform(x)
feature_mtx = StandardScaler().fit_transform(x)
feature_mtx

In [ ]:
# set number of clusters
kclusters = 10

# run k-means clustering
kmeans_2 = KMeans(n_clusters=kclusters, random_state=0).fit(feature_mtx)

# check cluster labels generated for each row in the dataframe
kmeans_2.labels_[0:10] 


In [ ]:
# Keep only Chinese Restaurant
nyc_grouped_2 = nyc_grouped.filter(['Neighborhood','Chinese Restaurant'], axis = 1)

In [ ]:
# add clustering labels
nyc_grouped_2.insert(0, 'Cluster Labels', kmeans_2.labels_)

nyc_merged_2 = neighborhoods

# merge nyc_merged with neighborhoods to add latitude/longitude for each neighborhood
nyc_merged_2 = nyc_merged_2.join(nyc_grouped_2.set_index('Neighborhood'), on='Neighborhood')

nyc_merged2.head() # check the last columns!

In [ ]:
nyc_merged2.groupby(by=['Borough']).mean().sort_values(by = 'Chinese Restaurant')